# Functions

#### Import packages and data

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime    
from matplotlib.pyplot import figure
from datetime import date, timedelta, datetime
import warnings
import pickle 
import ipywidgets
from ipypb import irange, track
import operator
from operator import itemgetter
pd.set_option('display.max_columns', 500) 
pd.set_option('display.max_rows', 500) 
plt.style.use(["seaborn-dark-palette", "seaborn-whitegrid"])


warnings.filterwarnings('ignore')
#Bitmex
with open("/Users/paolo/Desktop/Thesis_code/pickles/df_XBTUSD_bitmex.pickle", 'rb') as f:
    df_xbt_bitmex = pickle.load(f)
with open("/Users/paolo/Desktop/Thesis_code/pickles/df_ETHUSD_bitmex.pickle", 'rb') as f:
    df_eth_bitmex = pickle.load(f)
with open("/Users/paolo/Desktop/Thesis_code/pickles/df_XRPUSD_bitmex.pickle", 'rb') as f:
    df_xrp_bitmex = pickle.load(f)
    
#Binance
with open("/Users/paolo/Desktop/Thesis_code/pickles/df_XBTUSD_binance.pickle", 'rb') as f:
    df_xbt_binance = pickle.load(f)
with open("/Users/paolo/Desktop/Thesis_code/pickles/df_ETHUSD_binance.pickle", 'rb') as f:
    df_eth_binance = pickle.load(f)
with open("/Users/paolo/Desktop/Thesis_code/pickles/df_XRPUSD_binance.pickle", 'rb') as f:
    df_xrp_binance = pickle.load(f)

#Liquidations
with open("/Users/paolo/Desktop/Thesis_code/pickles/liquidation_data_xbt.pickle", 'rb') as f:
    liquidation_data_xbt = pickle.load(f)
with open("/Users/paolo/Desktop/Thesis_code/pickles/liquidation_data_eth.pickle", 'rb') as f:
    liquidation_data_eth = pickle.load(f)
with open("/Users/paolo/Desktop/Thesis_code/pickles/liquidation_data_xrp.pickle", 'rb') as f:
    liquidation_data_xrp = pickle.load(f)

#### How to save a pickle

In [3]:
def Save_pickle(name_model, what_to_save):
    name_model = name_model +".pickle"
    with open(name_model, 'wb') as f:
        pickle.dump(what_to_save, f)       

#### Cumulative return

In [4]:
def return_false(a=0,b=0,c=0,d=0,e=0):
    return False

In [5]:
def cumulative_return(vector_returns):
    
    vector_returns = pd.DataFrame(vector_returns)
    where_are_NaNs = np.isnan(vector_returns)
    vector_returns[where_are_NaNs] = 0
    
    #Calculate cumulative from daily
    cumulative = []
    return_sum = 0
    for i in (range(len(vector_returns))):
        return_sum = (return_sum + vector_returns.iloc[i])
        cumulative.append(return_sum)
    cumulative = pd.DataFrame(cumulative, index=vector_returns.index)
    return(cumulative)

In [6]:
def cumulative_step(df,variables=["fundingRate"],windows=[3], freq=1):
    
    for variable in variables:
        for window in windows:
            name = "cum_%s_%s" % (variable , window)

            df[name] = 0
            for i in track(range(len(df))):
                if df[variable][i]!=0:
                    df[name].iloc[i] = round(sum(df[variable][i-window*freq:i]),5)

    return df

#### Window return

In [7]:
def window_return(df, windows, skips, asset):
    if skips==0: 
        skips=windows
        
    for skip in skips:
        for window in windows:
            if(window>=skip):
                print(asset,window,skip)

                name = "return_%s_min_cum_each_%s" % (window,skip)
                df[name] = 0
                lista = []
                i= window
                while i < len(df)-window:
                    zeros = np.zeros(skip)
                    zeros[skip-1] = df["open"][i] / df["open"][i-window] -1
                    lista.append(zeros)
                    i = i + skip
                lista = np.concatenate(np.asarray(lista))

                df.loc[:len(lista),name] = lista
    return(df)

#### Analyze the trades

In [8]:
def Print_percent(something,rounded=3):
    return ( "%s %%" % round(100*(something),rounded) )

In [9]:
def first_condition_verified(df,column,condition,operator=operator.gt):
    test = lambda c: c == True
    data = operator(df[column],condition)
    #print(data)
    position = next(filter(itemgetter(1), enumerate(map(test, data))))[0] 
    result = df.iloc[position].name
    return result

In [10]:
def Analysis(trades): 
    configuration     = trades["configuration"][0]
    period_of_analysis =  "from %s to %s" % ( min(trades["date"].dt.to_pydatetime()), 
                                              max(trades["date"].dt.to_pydatetime())) #https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.to_pydatetime.html)
    n_possible_trades = len(trades)
    trading_days      = (trades.date.max() -trades.date.min()).days
    n_trades          = (trades["status"]=="Opened").sum()
    n_longs           = len(trades[(trades["status"]=="Opened") & (trades["direction"]=="Long")])
    n_shorts          = len(trades[(trades["status"]=="Opened") & (trades["direction"]=="Short")])
    return_price      = Print_percent(trades["realized_return"].sum())
    return_rate       = Print_percent(trades["collected_rate"].sum())
    return_rate_hold  = Print_percent(trades["collected_rate_hold"].sum())
    return_commission_open = Print_percent(trades["commission_open"].sum())
    return_commission_close = Print_percent(trades["commission_close"].sum())
    total_return      = Print_percent(trades["return_from_trade"].sum())
    trades["date_day"] = [str(trades["date"][i])[:-9] for i in range(len(trades))]
    trades_by_day     = trades.groupby(['date_day']).sum()
    ret_annual_internal  = trades_by_day["return_from_trade"].sum() *365 / trading_days
    ret_annual           = Print_percent(ret_annual_internal)
    corr_return_rate  = np.corrcoef(trades["realized_return"],trades["collected_rate"])[0][1]
    st_dev_internal   = np.asarray(trades_by_day["return_from_trade"]).std() *365**0.5
    st_dev            = Print_percent(st_dev_internal)
    sharpe_ratio      = ret_annual_internal/st_dev_internal 
    winning_trades    = Print_percent((trades["return_from_trade"]>0).sum() / max(n_trades,1))
    win_shorts        = len(trades[(trades["return_from_trade"]>0) & (trades["direction"]=="Short")]) / max(n_shorts,1)
    win_longs         = len(trades[(trades["return_from_trade"]>0) & (trades["direction"]=="Long")]) / max(n_longs,1)
    win_shorts        = Print_percent(win_shorts)
    win_longs         = Print_percent(win_longs)
    trade_opened_percent = Print_percent(n_trades / max(len(trades),1))
    take_profits_percent = Print_percent(len(trades[trades["type_of_close"]=="Take profit"])/n_trades)
    stop_loss_percent = Print_percent(len(trades[trades["type_of_close"]=="Stop loss"])/n_trades)
    holding_percent   = Print_percent(len(trades[trades["type_of_close"]=="After holding time"])/n_trades)
    holding_period    = round(trades[trades["status"]=="Opened"]["holding_period"].mean())
    holding_period_hour = round(holding_period/60,2)
     
    results =     [configuration,period_of_analysis,n_possible_trades,n_trades,n_longs,n_shorts, return_price, return_rate,return_rate_hold,  return_commission_open,
                    return_commission_close, total_return, ret_annual,corr_return_rate,st_dev,sharpe_ratio, winning_trades, win_shorts, win_longs, trade_opened_percent,
                  take_profits_percent,stop_loss_percent,holding_percent,holding_period, holding_period_hour]

    results = pd.DataFrame([results], columns=["configuration","Analysis period","Possible trades","Number of trades made ","Number of longs","Number of shorts" ,"Return from price change","Return from collected rate",
                                             "Return from collected rate hold position", "Return from commission open","Return from commission close", 
                                             "Total Return","Annual return","Correlation (Price, rate)","Annual Std","Sharpe ratio","Percent Win",
                                              "Percent Win Shorts","Percent Win Longs","Percent of trades opened","Percent of take profits","Percent of stop losses","Percent of close after hold",
                                               "Avg Holding period (minutes)","Avg Holding period (hours)"])
    return(results)

In [11]:
import matplotlib.ticker as mtick

def Plot_results(trades, asset, show_bars=False, show_cumulative=False, decompose=True, print_config=True):
    # [#001C7F, #017517, #8C0900, #B8860B, #006374]
    if print_config:print(trades["configuration"][0])
    figure(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
    ax = plt.subplot(111) 
    
    plt.hlines(0, xmin= min(trades["date"]), xmax=max(trades["date"]), linestyle="dashed",linewidth=1)
        
    if(show_cumulative): #You can put a minus and see how the 2 behaves together
        if decompose == False:
            plt.plot(trades.date, trades['return_money_management'],color = '#001C7F',linewidth=2.0, label= "Net return")
    
        if decompose:
            ax.plot(trades.date, 100*trades['net_return'],color = '#001C7F',linewidth=1.5, label= "Net return")
            ax.plot(trades.date, 100*trades['cumulative_realized_return'],color = '#017517',linewidth=1.5, label="Return from price change")
            if(trades["cumulative_commission"]!=0).any(): 
                ax.plot(trades.date, 100*trades['cumulative_commission'],color = '#8C0900',linewidth=1.5, label="Return from commission")
            ax.plot(trades.date, 100*trades['cumulative_collected_rate'],color = '#B8860B',linewidth=1.5, label = "Return from funding rate")   
            ax.plot(trades.date, 100*trades['cumulative_collected_rate_hold'],color = '#006374',linewidth=1.5, label = "Return from subsequent funding rates")   
            ax.yaxis.set_major_formatter(mtick.PercentFormatter())
            ax.fill_between(trades.date, 0, 100*trades['net_return'], alpha=0.12, color='#001C7F')

    if(show_bars): 
        ax.bar(trades.date, trades["realized_return"], color='#001C7F', align='center', label="Return from price change")        
        ax.bar(trades.date, trades["collected_rate"], color='#8C0900', align='center', label="Collected rate")
        
    if show_cumulative: 
        y_pos = min(min(trades["cumulative_realized_return"]), min(trades["cumulative_commission"]))
    else: 
        y_pos = min(trades["realized_return"])
    
    n_trades = (trades["status"]=="Opened").sum()
    #plt.title("%s: Number of trades: %s" % (asset,n_trades))
    
    ax.tick_params(axis='x', rotation=45)
    ax.legend()
    plt.show()

In [12]:
def Vector_conditions(signal,param,operat=operator.gt):   #operator.lt      
    comp = operat
    
    conditions = []
    for k in range(len(signal)):
        conditions.append(comp(signal[k],param[k]))
    return all(conditions)      

#### Mean reversion strategy

In [13]:
def Strategy(asset,exchanger="bitmex",my_signal=["fundingRate"],  long_param=[0], short_param=[0],                        
    window_mm_open=5, window_hold=9, window_mm_close=10, window_open_wrt_signal=0, long=True, short=True,
    date_start="2010-01-01", n_days=0, size=0, improving_param_open= 0, improving_param_close= 0,
    commission_market_making = 0.025/100, commission_market_taking = -0.075/100, divide=True,
    percent = False, equity_per_trade = 0.1, test=False,percent_split = 0.8, market_making=True, distribution_long_param = 0, distribution_short_param=0,
    debug=False, strategy_type="mean_reversion",break_max=False, close_condition="time",max_hold_position=30*24*60,
    condition_tp = "ongoing_return", parameter_tp = [0.02,-0.02], check_tp = [max,min], operator_tp = [return_false, return_false],
    condition_sl = "log_ret", parameter_sl = [-0.02,0.02], check_sl = [min,max], operator_sl = [return_false, return_false]
    ): 
    # Can make distribution_long_param to list
    # in the tp parameters, first is long, second is short
    # Important, if you want to put fundingRate, you need to insert it first
    # To avoid a TP or SL, insert "return false as an operator"
    #  condition_sl = "log_ret", parameter_sl = [-0.02,0.02], check_sl = [min,max], operator_sl = [operator.lt, operator.gt]
  
    # Locate correct dataset and load it 
    if exchanger =="bitmex":
        if asset   == "XBTUSD": df = df_xbt_bitmex
        elif asset == "ETHUSD": df = df_eth_bitmex
        elif asset == "XRPUSD": df = df_xrp_bitmex
        else: print("Asset should be one of %s" % assets)
    if exchanger =="binance":
        if asset   == "XBTUSD": df = df_xbt_binance
        elif asset == "ETHUSD": df = df_eth_binance
        elif asset == "XRPUSD": df = df_xrp_binance
        else: print("Asset should be one of %s" % assets)
     
    #Train and testset
    percent = round(len(df) * percent_split)
    if divide:
        if test: df = df.iloc[percent:]
        else:    df = df.iloc[:percent]
            
    if strategy_type =="mean_reversion":
        operator_long = operator.lt
        operator_short = operator.gt
    if strategy_type =="momentum":
        operator_long = operator.gt
        operator_short = operator.lt
        
    #Prepare some parameters
    if my_signal[0]=="fundingRate": 
        funding = True
        start_col = 4
    else: 
        funding=False
        start_col=5
    if (window_open_wrt_signal <0) and (window_mm_open >= abs(window_open_wrt_signal)) : 
        window_mm_open = abs(window_open_wrt_signal)-1
        
    if close_condition =="condition": window_hold = max_hold_position
    df["ongoing_return"]= 0
    
    #Reduce size of df
    n_signals = len(my_signal)
    my_col = df[my_signal]
    my_col2= df[[condition_tp,condition_sl]]
    df = df[["open","close","high","low","fundingRate"]]
    df = pd.concat([df, my_col], axis=1)
    df = pd.concat([df, my_col2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    df = df.loc[df.index > date_start]
    if n_days > 0:
        n_minutes = n_days*24*60
        df = df.head(n_minutes)
    
    # set distribution params
    if distribution_long_param >0:  
        long_param = np.nanpercentile(df[my_signal],distribution_long_param)
        print(long_param)
    if distribution_short_param >0: 
        short_param = np.nanpercentile(df[my_signal],distribution_short_param)
        print(short_param)
        
    trades = []
    max_signal = df[my_signal].max()
    
    # Create vector to check if position is open
    df["position_open"] = 0
    
    # Create yes or no for shorting / longing... # check one at a time
    for i in range(len(my_signal)):
        df["%s_long_cond" % my_signal[i]] = operator_long(df[my_signal[i]],long_param[i]) #smaller = operator_long
        df["%s_short_cond"% my_signal[i]] = operator_short(df[my_signal[i]],short_param[i])

    df_long     = df.filter(like="long_cond")
    df_short    = df.filter(like="short_cond")  
    long_signal = np.asarray((df_long.sum(axis=1))==len(my_signal))   #If I have all yesses  
    short_signal= np.asarray((df_short.sum(axis=1))==len(my_signal))
    
    if break_max:
        # Calculate Maximum relative
        max_rel = []
        check_window = 24*60
        i       = 0 
        while i < len(df):
            my_max = df["close"][(i-check_window):i].max()
            max_rel.append( np.array([my_max]*check_window) )
            i = i+check_window
        max_rel = [item for sublist in max_rel for item in sublist]
        df["relative_max"] = max_rel[:len(df)]
        
        df["long_cond_AboveMax"]    = df["close"] > df["relative_max"].shift(-1) #Maybe need a shift    
        
        long_signal = np.asarray(df["long_cond_AboveMax"])
        #print(long_signal.sum())
   
    for i in track(range(abs(window_open_wrt_signal) + 1, len(df)-(window_open_wrt_signal + window_mm_open + window_hold + window_mm_close) )):                        
        condition_for_short = short and short_signal[i]
        condition_for_long  = long  and long_signal[i]
             
        if condition_for_short or condition_for_long:            
            current_signal = np.asarray(df.iloc[i,start_col:start_col+n_signals]) #Maybe not here...

            #Set position size
            normalized    = abs(current_signal[0]  / max_signal[0])
            position_size = size * normalized + (1-size) # if size == 1, then position_size == normalized. if size == 0, then position_size == 1

            #Initialize logs variables
            data_for_opening= "NA"
            data_for_closing="NA"
            status = "Not opened"
            open_price_check = "NA" 
            position_closed = "NA"
            close_price_check ="NA"
            real_p_after = "NA"
            closed_at = "NA"
            commission_open = 0
            commission_close = 0
            rate_position = 0
            rate   = 0
            profit = 0
            position = "Short"

            #Timing
            date_of_signal   = df.index[i]  
            date_open_price  = date_of_signal + timedelta(0,window_open_wrt_signal*60) # for funding strat, this most likely is negative

            if market_making == False:
                status = "Opened" 
                position_closed = "Market taking"

                date_hold_end    = date_open_price  + timedelta(0,window_hold*60)
                date_close_price = date_hold_end    + timedelta(0,60) #I'll close at the open of following candle

                # Create the non overlapping periods
                p_open           = df["close"][df.index == date_open_price][0]          
                hold_position    = df[(df.index > date_open_price) & (df.index <=  date_hold_end)] #Do this for df
                p_after          = df["open"][df.index == date_close_price][0]     #Like this, I'm sure I'll close after 60 seconds

                #Short                                                     
                if condition_for_short:   #Short then long, collect the positive rate                   
                    commission_open = position_size * commission_market_taking    
                    rate_position   = position_size * hold_position["fundingRate"][abs(hold_position["fundingRate"])>0][1:].sum() #Exclude the first fundingrate
                    if funding: rate = position_size * current_signal[0] #Store first funding

                    profit       = round(position_size * np.log(p_open/p_after),4)
                    commission_close = position_size * commission_market_taking

                #Long                                         
                if condition_for_long:   #Long then short, collect the negative rate
                    position= "Long"    
                    commission_open = position_size * commission_market_taking                    
                    rate_position   = - position_size * hold_position["fundingRate"][abs(hold_position["fundingRate"])>0][1:].sum()

                    if funding: rate = -position_size * current_signal[0]

                    profit  = round(position_size * np.log(p_after/p_open),4)
                    commission_close = position_size * commission_market_taking


            if market_making:
                # Set the timing
                date_open_end    = date_open_price  + timedelta(0,window_mm_open*60)
                date_hold_end    = date_open_end    + timedelta(0,window_hold*60)
                date_close_price = date_hold_end    + timedelta(0,60)
                date_close_end   = date_close_price + timedelta(0,window_mm_close*60)

                # Create the non overlapping periods
                p_open           = df["close"][df.index == date_open_price][0]          
                data_for_opening = df[(df.index > date_open_price) & (df.index <=  date_open_end)]
                hold_position    = df[(df.index > date_open_end) & (df.index <=  date_hold_end)]
                p_after          = df["open"][df.index == date_close_price][0]     
                data_for_closing = df[(df.index > date_close_price) & (df.index <=  date_close_end)]

                #Short
                if condition_for_short:   #Short then long, collect the positive rate
                    p_open = p_open * (1+ improving_param_open)
                    open_price_check   = data_for_opening["high"].max()  #If that price is touched                

                    if(open_price_check > p_open):                     
                        status = "Opened"   
                        #df['position_open'][date_open_price] = 1
                        commission_open = position_size * commission_market_making                    
                        rate_position   = position_size * hold_position["fundingRate"][abs(hold_position["fundingRate"])>0][1:].sum()
                        if funding: rate = position_size * current_signal[0]
                        closed_at = "After holding time"                      
                        hold_position["ongoing_return"] = round(p_open/hold_position["close"] -1,4)

                        #Now the position is opened!  IF the condition to close is a condition, reset the times!                  
                        if close_condition == "condition":
                            tp_date, sl_date = datetime.now(),datetime.now() #Initialize the objects
                            close_tp = operator_tp[1](check_tp[1](hold_position[condition_tp]),parameter_tp[1])
                            close_sl = operator_sl[1](check_sl[1](hold_position[condition_sl]),parameter_sl[1])
                            if close_tp or close_sl: #If any condition on TP or SL is met
                                if close_tp: tp_date = first_condition_verified(hold_position,condition_tp, parameter_tp[1], operator_tp[1])
                                if close_sl: sl_date = first_condition_verified(hold_position,condition_sl, parameter_sl[1], operator_sl[1])
                                if(tp_date<sl_date): #If tp before sl
                                    condition_close, parameter_close, operator_close =  condition_tp, parameter_tp[1], operator_tp[1]
                                    closed_at = "Take profit"
                                else:
                                    condition_close, parameter_close, operator_close =  condition_sl, parameter_sl[1], operator_sl[1]
                                    closed_at = "Stop loss"
                                #Change dates accordingly
                                date_hold_end    = first_condition_verified(hold_position,condition_close, parameter_close, operator_close)
                                date_close_price = date_hold_end    + timedelta(0,60)
                                date_close_end   = date_close_price + timedelta(0,window_mm_close*60)

                                hold_position    = df[(df.index > date_open_end) & (df.index <=  date_hold_end)]
                                p_after          = df["close"][df.index == date_close_price][0]     
                                data_for_closing = df[(df.index > date_close_price) & (df.index <=  date_close_end)]

                        #Close the position at market making...
                        close_price_check  = data_for_closing["low"].min()
                        if improving_param_close > 0: p_after = p_after * (1- improving_param_close)

                        if (close_price_check < p_after):                            
                            position_closed = "Market making"
                            real_p_after = p_after
                            profit       = round(position_size * np.log(p_open/real_p_after),4)
                            commission_close = position_size * commission_market_making
                        else:
                            position_closed = "Market taking"
                            real_p_after = data_for_closing["close"][len(data_for_closing)-1] 
                            profit       = round(position_size * np.log(p_open/real_p_after),4)
                            commission_close = position_size * commission_market_taking


                #Long
                if condition_for_long:   #Long then short, collect the negative rate
                    p_open = p_open * (1- improving_param_open)
                    open_price_check = data_for_opening["low"].min()  #If that price is touched  
                    position = "Long"

                    if(open_price_check < p_open):
                        #df['position_open'][date_open_price] = 1
                        status="Opened"
                        commission_open = position_size * commission_market_making
                        rate_position   = -position_size * hold_position["fundingRate"][abs(hold_position["fundingRate"])>0][1:].sum()
                        if funding: rate = -position_size * current_signal[0]
                        closed_at = "After holding time"
                        hold_position["ongoing_return"] = round(hold_position["close"]/p_open -1 ,4)

                        if close_condition == "condition":
                            tp_date, sl_date = datetime.now(),datetime.now() #Initialize the objects
                            close_tp = operator_tp[0](check_tp[0](hold_position[condition_tp]),parameter_tp[0])
                            close_sl = operator_sl[0](check_sl[0](hold_position[condition_sl]),parameter_sl[0])
                            if close_tp or close_sl: #If any condition on TP or SL is met
                                if close_tp: tp_date = first_condition_verified(hold_position,condition_tp, parameter_tp[0], operator_tp[0])
                                if close_sl: sl_date = first_condition_verified(hold_position,condition_sl, parameter_sl[0], operator_sl[0])
                                if(tp_date<sl_date):
                                    condition_close, parameter_close, operator_close =  condition_tp, parameter_tp[0], operator_tp[0]
                                    closed_at = "Take profit"
                                else:
                                    condition_close, parameter_close, operator_close =  condition_sl, parameter_sl[0], operator_sl[0]
                                    closed_at = "Stop loss"
                                #Change dates accordingly
                                date_hold_end    = first_condition_verified(hold_position,condition_close, parameter_close, operator_close)
                                date_close_price = date_hold_end    + timedelta(0,60)
                                date_close_end   = date_close_price + timedelta(0,window_mm_close*60)

                                hold_position    = df[(df.index > date_open_end) & (df.index <=  date_hold_end)]
                                p_after          = df["close"][df.index == date_close_price][0]     
                                data_for_closing = df[(df.index > date_close_price) & (df.index <=  date_close_end)]

                        #Close the position at market making... 
                        if improving_param_close > 0: p_after = p_after * (1+ improving_param_close)
                        close_price_check  = data_for_closing["high"].max()

                        if(close_price_check > p_after):
                            position_closed = "Market making"
                            real_p_after = p_after
                            profit=round(position_size * np.log(real_p_after/p_open),4)
                            commission_close = position_size * commission_market_making
                        else:
                            position_closed = "Market taking"
                            real_p_after = data_for_closing["close"][len(data_for_closing)-1]
                            profit    = round(position_size * np.log(real_p_after/p_open),4)
                            commission_close = position_size * commission_market_taking

            #Debug stuff
            if(debug):
                print("signal:", current_signal)
                print("where is signal",df.iloc[i])
                print("p_open:",p_open)
                print("data for opening",data_for_opening)
                print("hold",hold_position)
                print("p_after:",p_after)
                print("data_for_closing:",data_for_closing)

            # Save the logs
            log ={"date":df.index[i],
                  "signal": current_signal,
                  "direction": position,
                  "status" : status,
                  "open_price": p_open,
                  "open_price_touched": open_price_check,
                  "want_to_close": p_after,
                  "close_price_touched": close_price_check,
                  "close price": real_p_after,
                  "position_closed": position_closed,
                  "collected_rate" : rate,
                  "collected_rate_hold": rate_position,
                  "commission_open":commission_open,
                  "commission_close":commission_close,
                  "realized_return":profit,
                  "type_of_close":closed_at,
                  "holding_period":len(hold_position)}
            
            trades.append(log)           
                #print(log)
    
    try: trades = pd.DataFrame(trades,columns=log.keys())
    except: print("No trade made: Error")
        
    trades["configuration"] = "asset=%s, my_signal=%s,long_param=%s,short_param=%s, window_mm_open=%s, window_hold=%s,window_mm_close=%s, window_open_wrt_signal=%s,long=%s, short=%s, date_start=%s,n_days=%s, size=%s, improving_param_open=%s, improving_param_close=%s, commission_market_making=%s, commission_market_taking=%s, percent=%s, equity_per_trade=%s, test=%s, market_making=%s, distribution_long_param=%s, distribution_short_param=%s, debug=%s, strategy_type=%s" % (asset, str(my_signal),  str(long_param), str(short_param), window_mm_open, window_hold, window_mm_close, window_open_wrt_signal, long, short, str(date_start), n_days, size, improving_param_open, improving_param_close, commission_market_making, commission_market_taking, percent, equity_per_trade, test, market_making, distribution_long_param, distribution_short_param, debug, strategy_type)
 
    # Calculate cumulatives
    trades["return_from_trade"] = trades["realized_return"] + trades["commission_open"] +trades["commission_close"] +trades["collected_rate"] +trades["collected_rate_hold"]  
    trades["cumulative_realized_return"] = cumulative_return(trades["realized_return"])
    trades["cumulative_commission"] = cumulative_return(trades["commission_open"]+trades["commission_close"])
    trades["cumulative_collected_rate"] = cumulative_return(trades["collected_rate"])
    trades["cumulative_collected_rate_hold"] = cumulative_return(trades["collected_rate_hold"])
    trades["net_return"] = trades["cumulative_realized_return"] + trades["cumulative_commission"] + trades["cumulative_collected_rate_hold"]+  trades["cumulative_collected_rate"]
    
    trades  = Money_management(trades, percent = percent, equity_per_trade = equity_per_trade)   
    results = Analysis(trades)  

    return results, trades 

#### Merge strategies

In [167]:
def Merge_strategies(list_of_trades, percent=False, equity_per_trade=0):
    my_df = list_of_trades[0].set_index("date")
    
    #From the second one...
    for trades_df in list_of_trades[1:]:
        to_add = trades_df.set_index("date")
        my_df = my_df.add(to_add,fill_value=0)
    
    #Recalculate cumulative...
    my_df["cumulative_realized_return"] = cumulative_return(my_df["realized_return"])
    my_df["cumulative_collected_rate"] = cumulative_return(my_df["collected_rate"])
    my_df["cumulative_collected_rate_hold"] = cumulative_return(my_df["collected_rate_hold"])
    my_df["cumulative_commission"] = cumulative_return(my_df["commission_open"]+my_df["commission_close"])
    my_df["net_return"]= my_df["cumulative_realized_return"] + my_df["cumulative_collected_rate"] + my_df["cumulative_commission"] + my_df["cumulative_collected_rate_hold"]

    #Set back date as a column
    my_df["date"] = my_df.index
    my_df = Money_management(my_df, percent=percent, equity_per_trade=equity_per_trade)

    return my_df

#### Money management

In [2]:
def Money_management(df, percent=False, equity_per_trade=0.2):
    
    if percent :
        df["net_return_coef"] = 1+ equity_per_trade*df["return_from_trade"]

        cumulative = []
        balance = 1
        for i in (range(len(df))):
            balance = (balance * df["net_return_coef"].iloc[i])
            cumulative.append(balance)

        df["return_money_management"] = cumulative
        df["return_money_management"] = df["return_money_management"]-1
        
    else:
        df["return_money_management"] = df["net_return"]
        
    return df

## Autocorrelation

In [198]:
def candle_price(my_df,candle,name, time="2018-08-03 00:00:00" ):
    df = my_df[my_df.index>time]
    df = df[["close"]]
    if(candle==60):   df = df[df.index.minute==0] #Hourly
    if(candle==60*24):df = df[(df.index.minute==0) & (df.index.hour==0)] #Daily
    df["return"] = np.log(df["close"] / df["close"].shift(1))
    df["return"] = np.nan_to_num(df["return"])
    df.columns = ["close_%s"% name ,"return_%s"% name]
    return(df)

In [ ]:
def normalize(df): #https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result